https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb

some deviations from the source code because i dont wanna pay for embeddings from openai, or hit openai models. All openAI integration is replaced with ollama.

I also removed langsmith integration. don't think it's needed. just a frontend for LLM debugging which i can achieve with `langchain.debug = True`

Try to roll my own adaptive RAG for my own use case. I already know how to use langgraph so time to practice. Don't wanna implement the same graph as what's done in the video cause i don't wanna do do a websearch.

Question --> Query Analysis --> Related to documents --> Vectorstore --> Relevant to question? --Yes--> LLM Generation --> Hallucination?
                                                                                   | No  
                                                                                   v  
                                                                                 HyDE  
                                                                                   |  
                                                                                   v  
                                                                              LLM Generation --> Hallucination?  

                               Query Analysis --> Anything else --> LLM Generation --> Hallucination?  

Hallucination? --Yes--> LLM Generation  
                 | No  
                 v  
          Answer Question? --Yes--> Give LLM response (End)  
                          | No  
                          v  
                   Give default response (End)  



The states i'll need:
1. question
2. documents (if vector store is hit)
3. generation

In [6]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [3]:
from typing_extensions import TypedDict
from typing import List

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        documents: documents
        generation: LLM generation
    """
    question: str
    generation: str
    documents: List[str]

In [2]:
# Load docs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding= OllamaEmbeddings(
        model="nomic-embed-text:v1.5"
    ),
)
retriever = vectorstore.as_retriever()

In [ ]:
def query_analysis(state):
    # Data model
    class web_search(BaseModel):
        """
        The internet. Use web_search for questions that are related to anything else than agents, prompt engineering, and adversarial attacks.
        """
        query: str = Field(description="The query to use when searching the internet.")

    class vectorstore(BaseModel):
        """
        A vectorstore containing documents related to agents, prompt engineering, and adversarial attacks. Use the vectorstore for questions on these topics.
        """
        query: str = Field(description="The query to use when searching the vectorstore.")

    question = state['question']
    llm_router = ChatOllama(
        model="command-r7b",
        temperature=0.1,
    )
    preamble="""You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
    llm_router.bind_tools(
        tools=[web_search, vectorstore], preamble=preamble
    )
    route = llm_router.invoke(question)

    # Route accordingly
    if 'tool_calls' not in route.additional_kwargs:
        raise "Router could not decide source"
    if len(route.additional_kwargs["tool_calls"]) == 0:
        raise "Router could not decide source"
    
    path = route.additional_kwargs['tool_calls'][0]['function']['name']
    if path == 'web_search':
        print("Not implementing websearch. End here.")
        return
    
    elif path == 'vectorstore':
        print("Routing to vectorstore")
        return 'vectorstore'
    
    else:
        print('Route to llm generation')
        return 'llm_generation'
